In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import time

In [19]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Data Loading
data_dir = '/content/drive/MyDrive/JPEGImages'
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),          # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 2. Data Preprocessing is included in the transforms.Compose


In [20]:
# 3. Model Definition
model = models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 50)  # Adjust to 50 classes
model = model.to(device)


In [21]:
# 4. Model Training with timing and periodic loss logging
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        start_time = time.time()
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if (i + 1) % 20 == 0:
                print(f'Step {i+1}, Loss: {loss.item()}')

        end_time = time.time()
        print(f'Epoch {epoch+1} completed in {end_time - start_time:.2f} seconds')

In [22]:
# 5. Model Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')


In [23]:
# Hyperparameters and loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate
train_model(model, train_loader, criterion, optimizer, num_epochs=10)
evaluate_model(model, test_loader)

Step 20, Loss: 2.9951627254486084
Step 40, Loss: 2.799605131149292
Step 60, Loss: 2.391989231109619
Step 80, Loss: 2.17104434967041
Step 100, Loss: 2.395364284515381
Step 120, Loss: 1.9272197484970093
Step 140, Loss: 2.289379835128784
Step 160, Loss: 1.6195214986801147
Step 180, Loss: 1.6020917892456055
Step 200, Loss: 1.7582021951675415
Step 220, Loss: 1.8844441175460815
Step 240, Loss: 1.9586910009384155
Epoch 1 completed in 39.92 seconds
Step 20, Loss: 1.5239369869232178
Step 40, Loss: 1.4006084203720093
Step 60, Loss: 1.592103123664856
Step 80, Loss: 1.6024843454360962
Step 100, Loss: 1.261006236076355
Step 120, Loss: 1.4093478918075562
Step 140, Loss: 1.229493260383606
Step 160, Loss: 1.0211800336837769
Step 180, Loss: 1.561867356300354
Step 200, Loss: 0.829927921295166
Step 220, Loss: 1.3483895063400269
Step 240, Loss: 1.386629581451416
Epoch 2 completed in 39.44 seconds
Step 20, Loss: 0.9723162055015564
Step 40, Loss: 0.5917725563049316
Step 60, Loss: 0.9242250323295593
Step 80,